<a href="https://colab.research.google.com/github/pedrossntos/saldo-licitacoes/blob/main/saldo_licitacoes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import pandas as pd
from bs4 import BeautifulSoup
from openpyxl import Workbook, load_workbook
from openpyxl.styles import Font, PatternFill, Alignment

In [ ]:
def adiciona_linha_excel(linha, arquivo, cabecalho_fornecedor=False):
    try:
        wb = load_workbook(arquivo)
        ws = wb.active
    except:
        wb = Workbook()
        ws = wb.active

    if cabecalho_fornecedor:     # Adicionar formatação depois
        for cell in ws[ws.max_row]:
            cell.font = Font(bold=True if cell.column == 1 else False)

    ws.append(linha)
    wb.save(arquivo)

In [ ]:
nome_arquivo = '119.24.htm' # ALTERAR O NOME DO ARQUIVO COM O RELATORIO DO EPROCESSOS AQUI
with open(nome_arquivo, encoding='latin-1') as saldo:
  saldo = saldo.read()
soup = BeautifulSoup(saldo, 'html.parser')

In [ ]:
# Encontrar todas as divs
divs = soup.find_all('div')
dict_saldo = {'fornecedor': [],
              'produto': [],
              'un_medida': [],
              'valor': [],
              'qtd': []}
# Iterar pelas divs para encontrar a primeira com "left: 82px" e conteúdo não vazio
for i, div in enumerate(divs):
    if 'left: 82px' in div.get('style', '') and not div.text.strip():
        fornecedor = divs[i + 3] if i + 3 < len(divs) else None
    elif 'left: 82px' in div.get('style', '') and div.text.strip():
        # Selecionar as próximas divs
        produto = divs[i + 11] if i + 11 < len(divs) else None
        un_medida = divs[i + 13] if i + 13 < len(divs) else None
        # Verifica se é fim de pagina
        if 'empenho' in produto.text.strip():
          produto = divs[i + 69] if i + 69 < len(divs) else None
          un_medida = divs[i + 71] if i + 71 < len(divs) else None
        valor = divs[i + 1] if i + 1 < len(divs) else None
        qtd = divs[i + 2] if i + 2 < len(divs) else None



        dict_saldo['fornecedor'].append(fornecedor.text.strip())
        dict_saldo['produto'].append(produto.text.strip())
        dict_saldo['un_medida'].append(un_medida.text.strip())
        dict_saldo['valor'].append(valor.text.strip())
        dict_saldo['qtd'].append(qtd.text.strip())

df_saldo = pd.DataFrame(dict_saldo)
df_saldo.head(20)


,fornecedor,produto,un_medida,valor,qtd
0,COUROS DO BOTINEIRO LTDA,"27368 Botina de segurança para eletricista, mo...",PAR,"74,00","5,00"
1,COUROS DO BOTINEIRO LTDA,27415 Óculos de segurança (proteção contra imp...,UN,"2,80","5,00"
2,COUROS DO BOTINEIRO LTDA,42109 Pedestal zebrado leve para delimitação e...,UN,"24,00","8,00"
3,COUROS DO BOTINEIRO LTDA,"27420 Protetor auricular, tipo concha, acolcho...",UN,"44,00","5,00"
4,COUROS DO BOTINEIRO LTDA,45434 Tênis de segurança confeccionado 100% em...,PAR,"100,00","8,00"
5,NACIONAL PROTEÇÃO LTDA,26778 Corrente de plástico com elos grandes (9...,M,"3,50","20,00"
6,COMERCIAL TOLOTTI LTDA EPP,"46055 Bloqueador/protetor solar, tipo loção, F...",UN,"11,63","40,00"
7,COMERCIAL TOLOTTI LTDA EPP,"14151 Repelente de insetos (mosquitos, pernilo...",FR,"10,03","100,00"
8,DATA EQUIPAMENTOS DE SEGURANÇA LTDA -,"45208 Capa de chuva, com capuz, confeccionada ...",UN,"13,50","20,00"
9,DATA EQUIPAMENTOS DE SEGURANÇA LTDA -,"27376 Cinturão de segurança, tipo paraquedista...",UN,"300,00","1,00"


In [ ]:
if os.path.exists(nome_arquivo+'.xlsx'):
    print('arquivo ja existe, apagando para nao dar merge com a planilha existente (execute novamente)...')
    os.remove(nome_arquivo+'.xlsx')
else:
    fornecedor_ex = ''
    for index, row in df_saldo.iterrows():
        if not row['fornecedor'] == fornecedor_ex:
            fornecedor_ex = row['fornecedor']
            adiciona_linha_excel([fornecedor_ex], nome_arquivo+'.xlsx')
            linha = [row['produto'], row['un_medida'], row['valor'], row['qtd']]
            adiciona_linha_excel(linha, nome_arquivo+'.xlsx', cabecalho_fornecedor=True)
        else:
            linha = [row['produto'], row['un_medida'], row['valor'], row['qtd']]
            adiciona_linha_excel(linha, nome_arquivo+'.xlsx')
    print('planilha criada com sucesso')

planilha criada com sucesso
